# Initialize `lamindb/bionty-assets` instance

In [1]:
!lamin load laminlabs/bionty-assets

💡 connected lamindb: laminlabs/bionty-assets


In [2]:
import lamindb as ln
import bionty as bt
from lamin_utils import logger


ln.settings.transform.stem_uid = "b8kF9q3jL3Ty"
ln.settings.transform.version = "2"
ln.track()

💡 connected lamindb: laminlabs/bionty-assets
💡 notebook imports: bionty==0.48.0 lamin_utils==0.13.2 lamindb==0.76.0
💡 saved: Transform(uid='b8kF9q3jL3Ty65cN', version='2', is_latest=True, name='Initialize `lamindb/bionty-assets` instance', key='initial-lamindb-instance', type='notebook', created_by_id=1, updated_at='2024-08-14 13:19:04 UTC')
💡 saved: Run(uid='yRzMupNp94cZQHAmeiCI', transform_id=4, created_by_id=1)


Run(uid='yRzMupNp94cZQHAmeiCI', started_at='2024-08-14 13:19:05 UTC', is_consecutive=True, transform_id=4, created_by_id=1)

In [3]:
bt.Source.df().head()

,uid,entity,organism,name,version,in_db,currently_used,description,url,md5,source_website,dataframe_artifact_id,run_id,created_by_id,updated_at
id,,,,,,,,,,,,,,,
63,7D6N,bionty.Phenotype,zebrafish,zp,2022-12-17,False,False,Zebrafish Phenotype Ontology,https://github.com/obophenotype/zebrafish-phen...,03430b567bf153216c0fa4c3440b3b24,https://github.com/obophenotype/zebrafish-phen...,109.0,NaN,1,2024-08-13 19:25:36.927718+00:00
27,3HPi,bionty.Protein,mouse,uniprot,2023-02,False,False,Uniprot,s3://bionty-assets/mouse_uniprot_2023-02_Prote...,dcae4f62f5df145a5c15163fce7e9135,https://www.uniprot.org,90.0,NaN,1,2024-08-13 19:24:58.002505+00:00
18,PGj9,bionty.Gene,mouse,ensembl,release-109,False,False,Ensembl,s3://bionty-assets/mouse_ensembl_release-109_G...,08a1165061151b270b985317322bd2ed,https://www.ensembl.org,85.0,NaN,1,2024-08-13 19:24:57.108181+00:00
29,7bV5,bionty.CellMarker,mouse,cellmarker,2.0,False,True,CellMarker,s3://bionty-assets/mouse_cellmarker_2.0_CellMa...,189586732c63be949e40dfa6a3636105,http://bio-bigdata.hrbmu.edu.cn/CellMarker,84.0,NaN,1,2024-08-13 19:24:56.210249+00:00
61,6Dzd,bionty.Phenotype,mammalian,mp,2023-05-31,False,False,Mammalian Phenotype Ontology,https://github.com/mgijax/mammalian-phenotype-...,be89052cf6d9c0b6197038fe347ef293,https://github.com/mgijax/mammalian-phenotype-...,108.0,NaN,1,2024-08-13 19:24:55.307867+00:00


In [4]:
ln.settings.storage.root.view_tree()

2 sub-directories & 176 files with suffixes '', '.0__ExperimentalFactor', '.84__Pathway', '.ipynb', '.txt', '.json', '.html', '.82__Pathway', '.parquet', '.csv', '.79__Pathway', '.0__Ethnicity', '.tsv.gz'
s3://bionty-assets/
├── all_cl_2022-08-16_CellType_lookup.parquet
├── all_cl_2023-02-15_CellType_lookup.parquet
├── all_cl_2023-04-20_CellType_lookup.parquet
├── all_clo_2022-03-21_CellLine_lookup.parquet
├── all_dron_2023-03-10_Drug_lookup.parquet
├── all_go_2023-05-10_Pathway_lookup.parquet
├── all_mondo_2022-10-11_Disease_lookup.parquet
├── all_mondo_2023-02-06_Disease_lookup.parquet
├── all_mondo_2023-04-04_Disease_lookup.parquet
├── all_pw_7.79_Pathway_lookup.parquet
├── all_uberon_2022-08-19_Tissue_lookup.parquet
├── all_uberon_2023-02-14_Tissue_lookup.parquet
├── all_uberon_2023-04-19_Tissue_lookup.parquet
├── bfxpipelines.json
├── df-legacy_human__ensembl__release-110__Gene.parquet
├── df-legacy_human__ensembl__release-111__Gene.parquet
├── df-legacy_human__ensembl__release-11

## Register DataFrames as artifacts

In [5]:
from bionty._bionty import list_biorecord_models

bt_models = list_biorecord_models(bt)

not_registered = {"no_source": [], "not_recognized": []}
for file in ln.settings.storage.root.glob("**/*"):
    filepath = file.path
    filestem = file.stem

    if file.is_dir():
        continue
    if filepath.startswith("bionty-assets/.lamindb"):
        continue

    if file.suffix == ".parquet":
        # new format
        if filestem.startswith("df_"):
            organism, source, version, entity = filestem.removeprefix("df_").split("__")
        # old format
        elif filestem.endswith("_lookup"):
            organism, source, version, entity = filestem.removesuffix("_lookup").split(
                "_"
            )
        else:
            not_registered["not_recognized"].append(file)
            logger.warning(f"Not recognized parquet file! {file}")
            continue
        if entity in bt_models:
            entity = f"bionty.{entity}"
        source_record = bt.Source.filter(
            organism=organism, name=source, version=version, entity=entity
        ).one_or_none()
        if source_record is not None:
            artifact = ln.Artifact(file).save()
            source_record.dataframe_artifact = artifact
            source_record.save()
            logger.print(f"Registered parquet file: {file}")
        else:
            not_registered["no_source"].append(file)
            logger.warning(f"Source not found for parquet file! {file}")
    elif file.stem.startswith("ontology_"):
        try:
            organism, source, version, entity = file.name.removeprefix(
                "ontology_"
            ).split("__")
        except ValueError:
            not_registered["not_recognized"].append(file)
            logger.warning(f"Not recognized file! {file}")
            continue
        if entity in bt_models:
            entity = f"bionty.{entity}"
        source_record = bt.Source.filter(
            organism=organism, name=source, version=version, entity=entity
        ).one_or_none()
        if source_record is not None:
            artifact = ln.Artifact(file).save()
            source_record.artifacts.add(artifact)
            logger.print(f"Registered ontology file: {file}")
        else:
            not_registered["no_source"].append(file)
            logger.warning(f"Source not found for ontology file! {file}")
    else:
        not_registered["not_recognized"].append(file)
        logger.warning(f"Not recognized file! {file}")

💡 returning existing artifact with same hash: Artifact(uid='e99CRxykvrACb4jBGHO5', is_latest=True, key='all_cl_2022-08-16_CellType_lookup.parquet', suffix='.parquet', size=222462, hash='bY0qmQrB4cOGStauW_Ihdw', _hash_type='md5', visibility=1, _key_is_virtual=False, created_by_id=1, storage_id=1, transform_id=2, run_id=2, updated_at='2024-08-13 19:23:15 UTC')
Registered parquet file: s3://bionty-assets/all_cl_2022-08-16_CellType_lookup.parquet
💡 returning existing artifact with same hash: Artifact(uid='AP5BJ7SIYrLrvtaXzeW5', is_latest=True, key='all_cl_2023-02-15_CellType_lookup.parquet', suffix='.parquet', size=231131, hash='XonVIyDPPZ9jPXxkfVOEcA', _hash_type='md5', visibility=1, _key_is_virtual=False, created_by_id=1, storage_id=1, transform_id=2, run_id=2, updated_at='2024-08-13 19:23:15 UTC')
Registered parquet file: s3://bionty-assets/all_cl_2023-02-15_CellType_lookup.parquet
💡 returning existing artifact with same hash: Artifact(uid='PevD1YVSOTXw8vP0oiLd', is_latest=True, key='al

## Non-registered files

In [6]:
not_registered

{'no_source': [],
 'not_recognized': [S3Path('s3://bionty-assets/bfxpipelines.json'),
  S3Path('s3://bionty-assets/bfxpipelines_data/bfxpipelines.json'),
  S3Path('s3://bionty-assets/bfxpipelines_data/custom_pipelines.json'),
  S3Path('s3://bionty-assets/bfxpipelines_data/nf_core_pipelines.json'),
  S3Path('s3://bionty-assets/df-legacy_human__ensembl__release-110__Gene.parquet'),
  S3Path('s3://bionty-assets/df-legacy_human__ensembl__release-111__Gene.parquet'),
  S3Path('s3://bionty-assets/df-legacy_human__ensembl__release-112__Gene.parquet'),
  S3Path('s3://bionty-assets/df-legacy_mouse__ensembl__release-110__Gene.parquet'),
  S3Path('s3://bionty-assets/df-legacy_mouse__ensembl__release-111__Gene.parquet'),
  S3Path('s3://bionty-assets/df-legacy_mouse__ensembl__release-112__Gene.parquet'),
  S3Path('s3://bionty-assets/icd_10_cm.csv_0_0_0.csv'),
  S3Path('s3://bionty-assets/terminology__icd_9_cm.csv'),
  S3Path('s3://bionty-assets/terminology__icd_9_pcs.csv'),
  S3Path('s3://bionty-as

## Sources that have no linked dataframe

In [7]:
bt.Source.filter(dataframe_artifact=None).df()

,uid,entity,organism,name,version,in_db,currently_used,description,url,md5,source_website,dataframe_artifact_id,run_id,created_by_id,updated_at
id,,,,,,,,,,,,,,,


In [8]:
ln.finish()

✅ cell execution numbers increase consecutively
💡 go to: https://lamin.ai/laminlabs/bionty-assets/transform/b8kF9q3jL3Ty65cN
💡 if you want to update your notebook without re-running it, use `lamin save notebook.ipynb`
